In [45]:
import gym
import numpy as np
import tensorflow as tf
import random
from __future__ import print_function

print(tf.__version__) # running 0.9 right now unfortunately, will look for a newer docker machine myself or set it up

0.9.0


In [15]:
env = gym.make('FrozenLake-v0')

tf.reset_default_graph()

#input is a column
inputs = tf.placeholder(shape=[None,env.observation_space.n], dtype=tf.float32)

# Xavier initialization helps signals reach deep into the network
# Check the difference between tensorflow variable and tensorflow get variable
W = tf.get_variable(name="W", dtype=tf.float32, shape=[env.observation_space.n, env.action_space.n], initializer = tf.contrib.layers.xavier_initializer())

# This is a lower level way of accessing tensorflow variables
# better to use 
b = tf.Variable(tf.zeros(shape=[env.action_space.n]))

# We get the predicted Q out by multiplying our input with the weights of the network and then adding a bias term
qpred = tf.add(tf.matmul(inputs,W),b)

# Argmax over actions - first axis
apred = tf.argmax(qpred, 1)

#qtar = target value for Q
qtar = tf.placeholder(shape=[1, env.action_space.n], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(qtar-qpred))

[2018-10-13 21:40:17,000] Making new env: FrozenLake-v0


In [16]:
train = tf.train.AdamOptimizer(learning_rate=0.001)
minimizer = train.minimize(loss)

In [46]:
# Training the neural network

init = tf.initialize_all_variables()
y = 0.5 #learning rate
e = 0.3 #epsilon for greedy epsilon search strategy
episodes = 10000


#list to capture total steps and rewards per episodes
slist = []
rlist = []


In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(episodes):
        s = env.reset()
        r_total = 0
        while(True):
            a_pred,q_pred = sess.run([apred,qpred],feed_dict={inputs:np.identity(env.observation_space.n)[s:s+1]}) #??
            if np.random.uniform(low=0,high=1) < e:
                a_pred[0] = env.action_space.sample()            
            s_,r,t,_ = env.step(a_pred[0])  #action taken and new state 's_' is encountered with a feedback reward 'r'
            if r==0: 
                if t==True:
                    r=-5  #if hole make the reward more negative
                else:
                    r=-1  #if block is fine/frozen then give slight negative reward to optimise the path
            if r==1:
                    r=5       #good positive goat state reward

            q_pred_new = sess.run(qpred,feed_dict={inputs:np.identity(env.observation_space.n)[s_:s_+1]})

            #update the Q-target value for action taken
            targetQ = q_pred
            max_qpredn = np.max(q_pred_new)
            targetQ[0,a_pred[0]] = r + y*max_qpredn
            #this gives our targetQ

            _ = sess.run(minimizer,feed_dict={inputs:np.identity(env.observation_space.n)[s:s+1],qtar:targetQ})
            r_total+=r

            s=s_
            if t==True:
                break
    

    s = env.reset()
    env.render()
    while(True):
        a = sess.run(apred,feed_dict={inputs:np.identity(env.observation_space.n)[s:s+1]})
        s_,r,t,_ = env.step(a[0])
        env.render()
        s = s_
        if t==True:
            break


0.9.0
